# Directory settings

6つのexp ごった煮版

ハイパラチューン済み

In [ ]:
# ====================================================
# Directory settings
# ====================================================
import os

INPUT_DIR = '../input/us-patent-phrase-to-phrase-matching/'
OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [ ]:
os.system('pip uninstall -y transformers')
os.system('pip uninstall -y tokenizers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheel-20220531 transformers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheel-20220531 tokenizers')



In [ ]:
# exp100

def exp100():

    # ====================================================
    # CFG
    # ====================================================
    class CFG:
        num_workers=0
        path="../input/uspppm-exp100/"
        config_path=path+'config.pth'
        model="google/electra-large-discriminator"
        ckpt_name="electra-large-discriminator"
        batch_size=16
        target_size=1
        max_len=404
        seed=42
        n_fold=4
        tar_token_id = 30522
        tar_token = '[TAR]'
        trn_fold=[0, 1, 2, 3]
        n_augs = 2
    # Library
    # ====================================================
    # Library
    # ====================================================
    import os
    import gc
    import re
    import ast
    import sys
    import copy
    import json
    import time
    import math
    import shutil
    import string
    import pickle
    import random
    import joblib
    import itertools
    from pathlib import Path
    import warnings
    warnings.filterwarnings("ignore")

    import scipy as sp
    import numpy as np
    import pandas as pd
    pd.set_option('display.max_rows', 500)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.width', 1000)
    from tqdm.auto import tqdm
    from sklearn.metrics import f1_score
    from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

    import torch
    print(f"torch.__version__: {torch.__version__}")
    import torch.nn as nn
    from torch.nn import Parameter
    import torch.nn.functional as F
    from torch.optim import Adam, SGD, AdamW
    from torch.utils.data import DataLoader, Dataset
    from torch import inference_mode

    import tokenizers
    import transformers
    print(f"tokenizers.__version__: {tokenizers.__version__}")
    print(f"transformers.__version__: {transformers.__version__}")
    from transformers import AutoTokenizer, AutoModel, AutoConfig
    from transformers import DataCollatorWithPadding
    from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
    %env TOKENIZERS_PARALLELISM=false #true

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Utils
    # ====================================================
    # Utils
    # ====================================================
    def get_score(y_true, y_pred):
        score = sp.stats.pearsonr(y_true, y_pred)[0]
        return score


    def get_logger(filename=OUTPUT_DIR+'train'):
        from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
        logger = getLogger(__name__)
        logger.setLevel(INFO)
        handler1 = StreamHandler()
        handler1.setFormatter(Formatter("%(message)s"))
        handler2 = FileHandler(filename=f"{filename}.log")
        handler2.setFormatter(Formatter("%(message)s"))
        logger.addHandler(handler1)
        logger.addHandler(handler2)
        return logger

    LOGGER = get_logger()

    def seed_everything(seed=42):
        random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True

    seed_everything(seed=42)
    # OOF
    oof_df = pd.read_pickle(CFG.path+'oof_df.pkl')
    labels = oof_df['score'].values
    preds = oof_df['pred'].values
    score = get_score(labels, preds)
    LOGGER.info(f'CV Score: {score:<.4f}')
    # Data Loading
    # ====================================================
    # Data Loading
    # ====================================================
    # test = pd.read_csv(INPUT_DIR+'train.csv').sample(n=100)
    test = pd.read_csv(INPUT_DIR+'test.csv')
    submission = pd.read_csv(INPUT_DIR+'sample_submission.csv')
    print(f"test.shape: {test.shape}")
    print(f"submission.shape: {submission.shape}")
    display(test.head())
    display(submission.head())
    # ====================================================
    # CPC Data
    # ====================================================
    cpc_texts = torch.load(CFG.path+"cpc_texts.pth")
    test['context_text'] = test['context'].map(cpc_texts)
    display(test.head())
    anchor_context_grouped_target = test.groupby(['anchor', 'context'])['target'].apply(list)
    anchor_context_grouped_id = test.groupby(['anchor', 'context'])['id'].apply(list)
    i = pd.DataFrame(anchor_context_grouped_id).reset_index()
    t = pd.DataFrame(anchor_context_grouped_target).reset_index()
    test = t.merge(i, on=['anchor', 'context'])
    test['context_text'] = test['context'].map(cpc_texts)
    test = test.rename(columns={'target': 'targets', 'id': 'ids'})
    test['n_ids'] = test['ids'].map(len)
    test.head()
    from functools import reduce

    def cat_ids(x) -> str:
        return reduce(lambda a,b: a+b, x)
    import random 

    cat_ids_sets = set(test['ids'].map(cat_ids))
    auged_test = test.copy()
    auged_rows = []
    for a in range(CFG.n_augs):
        for _,r in test.iterrows():
            if r.n_ids == 1:
                continue
            indices = list(range(r.n_ids))
            random.shuffle(indices)
            if cat_ids(list(np.array(r.ids)[indices])) in cat_ids_sets:
                continue
            auged_r = r.copy()
            auged_r.targets = list(np.array(auged_r.targets)[indices])
            auged_r.ids = list(np.array(auged_r.ids)[indices])
            cat_ids_sets.add(cat_ids(auged_r.ids))
            auged_rows.append(auged_r)
    auged_test = test.append(
        pd.DataFrame(auged_rows, columns=test.columns).reset_index(drop=True))
    exploded_test = auged_test.explode(['targets', 'ids']).rename(columns={
            'targets': 'target',
            'ids': 'id',
        }).reset_index(drop=True)
    # tokenizer
    # ====================================================
    # tokenizer
    # ====================================================
    CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')
    assert CFG.tokenizer.encode(CFG.tar_token, add_special_tokens=False)[0] == CFG.tar_token_id
    assert CFG.tokenizer.decode(CFG.tar_token_id) == CFG.tar_token

    setattr(CFG.tokenizer, 'tar_token', CFG.tar_token)
    setattr(CFG.tokenizer, 'tar_token_id', CFG.tar_token_id)
    # Dataset
    # ====================================================
    # Dataset
    # ====================================================

    class TestDataset(Dataset):
        def __init__(self, cfg, df):
            self.cfg = cfg
            self.anchors = df['anchor'].values
            self.target_lists = df['targets'].values
            self.id_lists = df['ids'].values
            self.context_texts = df['context_text'].values
            self.max_len = cfg.max_len
            self.tokenizer = cfg.tokenizer

        def __len__(self):
            return len(self.id_lists)

        def __getitem__(self, item):
            target_mask = np.zeros(self.max_len)
            targets = np.array(self.target_lists[item])

            text = ''
            text += self.tokenizer.cls_token
            text += self.anchors[item]
            text += self.tokenizer.sep_token
            for target in targets:
                text += target + self.tokenizer.tar_token
            text += self.context_texts[item] + self.tokenizer.sep_token

            encoded = self.tokenizer(
                text,
                max_length = self.max_len,
                padding='max_length',
                add_special_tokens=False,
                truncation=True
            )

            cnt_tar = 0
            cnt_sep = 0
            nth_target = -1
            prev_i = -1

            for i, input_id in enumerate(encoded['input_ids']):
                if input_id == self.tokenizer.tar_token_id:
                    cnt_tar += 1
                    if cnt_tar == len(targets):
                        break
                if input_id == self.tokenizer.sep_token_id:
                    cnt_sep += 1

                if cnt_sep == 1 and input_id not in [self.tokenizer.pad_token_id, self.tokenizer.sep_token_id, self.tokenizer.tar_token_id]:
                    if (i-prev_i) > 1:
                        nth_target += 1
                    target_mask[i] = 1
                    prev_i = i

            for k,v in encoded.items():
                encoded[k] = torch.tensor(v, dtype=torch.long)

            return encoded, target_mask
    # Model
    # ====================================================
    # Model
    # ====================================================
    class CustomModel(nn.Module):
        def __init__(self, cfg, config_path=None, pretrained=False, n_vocabs=0):
            super().__init__()
            self.cfg = cfg
            if config_path is None:
                self.config = AutoConfig.from_pretrained(
                    cfg.model, output_hidden_states=True)
            else:
                self.config = torch.load(config_path)
            if pretrained:
                self.model = AutoModel.from_pretrained(
                    cfg.model, config=self.config)
            else:
                self.model = AutoModel.from_config(self.config)
            self.model.resize_token_embeddings(n_vocabs)
            self.fc = nn.Linear(self.config.hidden_size, 1)

        def feature(self, inputs):
            outputs = self.model(**inputs)
            last_hidden_states = outputs[0]
            return last_hidden_states

        def forward(self, inputs):
            feature = self.feature(inputs)
            output = self.fc(feature).squeeze(-1)
            return output
    # inference
    # ====================================================
    # inference
    # ====================================================
    @inference_mode()
    def inference_fn(test_loader, model, device):
        preds = []
        model.eval()
        model.to(device)
        for inputs, target_masks in test_loader:
            for k, v in inputs.items():
                inputs[k] = v.to(device)
            y_preds = model(inputs)
            y_preds = y_preds.sigmoid().to('cpu').numpy()

            anchorwise_preds = []
            for pred, target_mask, in zip(y_preds, target_masks):
                prev_i = -1
                targetwise_pred_scores = []
                for i, (p, tm) in enumerate(zip(pred, target_mask)):
                    if tm != 0:
                        if i-1 == prev_i:
                            targetwise_pred_scores[-1].append(p)
                        else:
                            targetwise_pred_scores.append([p])
                        prev_i = i
                for targetwise_pred_score in targetwise_pred_scores:
                    anchorwise_preds.append(np.mean(targetwise_pred_score))
            preds.append(anchorwise_preds)
        return preds
    # postprocess
    import numpy as np
    import pandas as pd

    import re
    import nltk
    from nltk.stem import PorterStemmer
    from nltk.stem import WordNetLemmatizer

    class PostProcess:
        def __init__(self, df, options={}):
            self.options = {
                'fit_near_score_th': 0.005,
                'normalizer': ('lemmatizer') # 'stemmer'
            }
            self.options.update(options)
            self.df = df.copy()
            self.predictions = []

        def add_predection(self, pred):
            # shape => (36, 1)
            self.predictions.append(pred.reshape(-1, 1))

        def nanmean(self, preds):
            return np.nanmean(preds, axis=0).reshape(-1)

        def equal_score_1(self, preds):
            """
            単純に anchor == target は score 1.0 にする
            """
            df = self.df.copy()
            df['preds'] = preds
            df.loc[df.anchor == df.target, ['preds']] = 1.0
            return df['preds'].values

        def normalize_score_1(self, preds):
            """
            normalizeした結果 == なら score を 1.0にする
            """
            df = self.df.copy()
            normalizer_options = self.options['normalizer']
            lemmatizer = WordNetLemmatizer()
            ps = PorterStemmer()
            use_lemmatizer = 'lemmatizer' in normalizer_options
            use_stemmer = 'stemmer' in normalizer_options
            def normalize(word):
                w = word.lower()
                if use_lemmatizer:
                    w = lemmatizer.lemmatize(w)
                if use_stemmer:
                    w = ps.stem(w)
                return w
            def normalize_words(word):
                words = re.split(r'\s+', word)
                words = [normalize(t) for t in words]
                return ' '.join(words)
            an = df.anchor.map(normalize_words)
            tn = df.target.map(normalize_words)
            df['preds'] = preds
            df.loc[an == tn, ['preds']] = 1.0
            return df['preds'].values

        def fit_near_score(self, preds):
            fit_n = self.options['fit_near_score_th']
            def fit_score(pred):
                if pred <= fit_n:
                    return 0.0
                elif pred >= (0.25 - fit_n) and pred <= (0.25 + fit_n):
                    return 0.25
                elif pred >= (0.5 - fit_n) and pred <= (0.5 + fit_n):
                    return 0.5
                elif pred >= (0.75 - fit_n) and pred <= (0.75 + fit_n):
                    return 0.75
                elif pred >= (1 - fit_n):
                    return 1.0
                return pred
            vect = np.vectorize(fit_score)
            return vect(preds)

        def get_score(self, apply_funcs=['nanmean', 'equal_score_1', 'normalize_score_1', 'fit_near_score']):
            preds = self.predictions
            for funcname in apply_funcs:
                fn = getattr(self, funcname)
                preds = fn(preds)
            return preds.reshape(-1, 1) # 提出形式に合わせる


    post_process = PostProcess(df=exploded_test)
    # predictions
    from functools import reduce

    test_dataset = TestDataset(CFG, auged_test)
    test_loader = DataLoader(test_dataset,
                             batch_size=CFG.batch_size,
                             shuffle=False,
                             num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
    for fold in CFG.trn_fold:
        model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False, n_vocabs=len(CFG.tokenizer))
        state = torch.load(CFG.path+f"{CFG.ckpt_name}_fold{fold}_best.pth",
                           map_location=torch.device('cpu'))
        model.load_state_dict(state['model'])
        prediction = inference_fn(test_loader, model, device)
        prediction = np.array(reduce(lambda a,b: a+b, prediction))
        post_process.add_predection(prediction)
        del model, state, prediction; gc.collect()
        torch.cuda.empty_cache()
    predictions = post_process.get_score()
    # Submission
    exploded_test['score']=predictions
    exploded_test = exploded_test.groupby('id').mean().reset_index() # aggregate auged samples
    submission_100 = exploded_test[['id','score']].sort_index()
    display(submission_100.head())
    return submission_100

In [ ]:
# 099
def exp099():
    # ====================================================
    # CFG
    # ====================================================
    class CFG:
        num_workers=0
        path="../input/uspppm-exp099/"
        config_path=path+'config.pth'
        model="microsoft/deberta-v3-large"
        ckpt_name="deberta-v3-large"
        batch_size=16
        target_size=1
        max_len=512
        seed=42
        n_fold=4
    #     trn_fold=[0]
        tar_token_id = 128001
        tar_token = '[TAR]'
        trn_fold=[0, 1, 2, 3]
        n_augs = 2

    # Library
    # ====================================================
    # Library
    # ====================================================
    import os
    import gc
    import re
    import ast
    import sys
    import copy
    import json
    import time
    import math
    import shutil
    import string
    import pickle
    import random
    import joblib
    import itertools
    from pathlib import Path
    import warnings
    warnings.filterwarnings("ignore")

    import scipy as sp
    import numpy as np
    import pandas as pd
    pd.set_option('display.max_rows', 500)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.width', 1000)
    from tqdm.auto import tqdm
    from sklearn.metrics import f1_score
    from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

    import torch
    print(f"torch.__version__: {torch.__version__}")
    import torch.nn as nn
    from torch.nn import Parameter
    import torch.nn.functional as F
    from torch.optim import Adam, SGD, AdamW
    from torch.utils.data import DataLoader, Dataset
    from torch import inference_mode

    # os.system('pip uninstall -y transformers')
    # os.system('pip uninstall -y tokenizers')
    # os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheel-20220531 transformers')
    # os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheel-20220531 tokenizers')

    import tokenizers
    import transformers
    print(f"tokenizers.__version__: {tokenizers.__version__}")
    print(f"transformers.__version__: {transformers.__version__}")
    from transformers import AutoTokenizer, AutoModel, AutoConfig
    from transformers import DataCollatorWithPadding
    from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
    %env TOKENIZERS_PARALLELISM=false #true

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Utils
    # ====================================================
    # Utils
    # ====================================================
    def get_score(y_true, y_pred):
        score = sp.stats.pearsonr(y_true, y_pred)[0]
        return score


    def get_logger(filename=OUTPUT_DIR+'train'):
        from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
        logger = getLogger(__name__)
        logger.setLevel(INFO)
        handler1 = StreamHandler()
        handler1.setFormatter(Formatter("%(message)s"))
        handler2 = FileHandler(filename=f"{filename}.log")
        handler2.setFormatter(Formatter("%(message)s"))
        logger.addHandler(handler1)
        logger.addHandler(handler2)
        return logger

    LOGGER = get_logger()

    def seed_everything(seed=42):
        random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True

    seed_everything(seed=42)
    # OOF
    oof_df = pd.read_pickle(CFG.path+'oof_df.pkl')
    labels = oof_df['score'].values
    preds = oof_df['pred'].values
    score = get_score(labels, preds)
    LOGGER.info(f'CV Score: {score:<.4f}')
    # Data Loading
    # ====================================================
    # Data Loading
    # ====================================================
    # test = pd.read_csv(INPUT_DIR+'train.csv').sample(n=100)
    test = pd.read_csv(INPUT_DIR+'test.csv')
    submission = pd.read_csv(INPUT_DIR+'sample_submission.csv')
    print(f"test.shape: {test.shape}")
    print(f"submission.shape: {submission.shape}")
    display(test.head())
    display(submission.head())
    # ====================================================
    # CPC Data
    # ====================================================
    cpc_texts = torch.load(CFG.path+"cpc_texts.pth")
    test['context_text'] = test['context'].map(cpc_texts)
    display(test.head())
    anchor_context_grouped_target = test.groupby(['anchor', 'context'])['target'].apply(list)
    anchor_context_grouped_id = test.groupby(['anchor', 'context'])['id'].apply(list)
    i = pd.DataFrame(anchor_context_grouped_id).reset_index()
    t = pd.DataFrame(anchor_context_grouped_target).reset_index()
    test = t.merge(i, on=['anchor', 'context'])
    test['context_text'] = test['context'].map(cpc_texts)
    test = test.rename(columns={'target': 'targets', 'id': 'ids'})
    test['n_ids'] = test['ids'].map(len)
    test.head()
    from functools import reduce

    def cat_ids(x) -> str:
        return reduce(lambda a,b: a+b, x)
    import random 

    cat_ids_sets = set(test['ids'].map(cat_ids))
    auged_test = test.copy()
    auged_rows = []
    
    for a in range(CFG.n_augs):
        for _,r in test.iterrows():
            if r.n_ids == 1:
                continue
            indices = list(range(r.n_ids))
            random.shuffle(indices)
            if cat_ids(list(np.array(r.ids)[indices])) in cat_ids_sets:
                continue
            auged_r = r.copy()
            auged_r.targets = list(np.array(auged_r.targets)[indices])
            auged_r.ids = list(np.array(auged_r.ids)[indices])
            cat_ids_sets.add(cat_ids(auged_r.ids))
            auged_rows.append(auged_r)
    auged_test = test.append(
        pd.DataFrame(auged_rows, columns=test.columns).reset_index(drop=True))
    exploded_test = auged_test.explode(['targets', 'ids']).rename(columns={
            'targets': 'target',
            'ids': 'id',
        }).reset_index(drop=True)
    # tokenizer
    # ====================================================
    # tokenizer
    # ====================================================
    CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')
    assert CFG.tokenizer.encode(CFG.tar_token, add_special_tokens=False)[0] == CFG.tar_token_id
    assert CFG.tokenizer.decode(CFG.tar_token_id) == CFG.tar_token

    setattr(CFG.tokenizer, 'tar_token', CFG.tar_token)
    setattr(CFG.tokenizer, 'tar_token_id', CFG.tar_token_id)
    # Dataset
    # ====================================================
    # Dataset
    # ====================================================

    class TestDataset(Dataset):
        def __init__(self, cfg, df):
            self.cfg = cfg
            self.anchors = df['anchor'].values
            self.target_lists = df['targets'].values
            self.id_lists = df['ids'].values
            self.context_texts = df['context_text'].values
            self.max_len = cfg.max_len
            self.tokenizer = cfg.tokenizer

        def __len__(self):
            return len(self.id_lists)

        def __getitem__(self, item):
            target_mask = np.zeros(self.max_len)
            targets = np.array(self.target_lists[item])

            text = ''
            text += self.tokenizer.cls_token
            text += self.anchors[item]
            text += self.tokenizer.sep_token
            for target in targets:
                text += target + self.tokenizer.tar_token
            text += self.context_texts[item] + self.tokenizer.sep_token

            encoded = self.tokenizer(
                text,
                max_length = self.max_len,
                padding='max_length',
                add_special_tokens=False,
                truncation=True
            )

            cnt_tar = 0
            cnt_sep = 0
            nth_target = -1
            prev_i = -1

            for i, input_id in enumerate(encoded['input_ids']):
                if input_id == self.tokenizer.tar_token_id:
                    cnt_tar += 1
                    if cnt_tar == len(targets):
                        break
                if input_id == self.tokenizer.sep_token_id:
                    cnt_sep += 1

                if cnt_sep == 1 and input_id not in [self.tokenizer.pad_token_id, self.tokenizer.sep_token_id, self.tokenizer.tar_token_id]:
                    if (i-prev_i) > 1:
                        nth_target += 1
                    target_mask[i] = 1
                    prev_i = i

            for k,v in encoded.items():
                encoded[k] = torch.tensor(v, dtype=torch.long)

            return encoded, target_mask
    # Model
    # ====================================================
    # Model
    # ====================================================
    class CustomModel(nn.Module):
        def __init__(self, cfg, config_path=None, pretrained=False, n_vocabs=0):
            super().__init__()
            self.cfg = cfg
            if config_path is None:
                self.config = AutoConfig.from_pretrained(
                    cfg.model, output_hidden_states=True)
            else:
                self.config = torch.load(config_path)
            if pretrained:
                self.model = AutoModel.from_pretrained(
                    cfg.model, config=self.config)
            else:
                self.model = AutoModel.from_config(self.config)
            self.model.resize_token_embeddings(n_vocabs)
            self.fc = nn.Linear(self.config.hidden_size, 1)

        def feature(self, inputs):
            outputs = self.model(**inputs)
            last_hidden_states = outputs[0]
            return last_hidden_states

        def forward(self, inputs):
            feature = self.feature(inputs)
            output = self.fc(feature).squeeze(-1)
            return output
    # inference
    # ====================================================
    # inference
    # ====================================================
    @inference_mode()
    def inference_fn(test_loader, model, device):
        preds = []
        model.eval()
        model.to(device)
        for inputs, target_masks in test_loader:
            for k, v in inputs.items():
                inputs[k] = v.to(device)
            y_preds = model(inputs)
            y_preds = y_preds.sigmoid().to('cpu').numpy()

            anchorwise_preds = []
            for pred, target_mask, in zip(y_preds, target_masks):
                prev_i = -1
                targetwise_pred_scores = []
                for i, (p, tm) in enumerate(zip(pred, target_mask)):
                    if tm != 0:
                        if i-1 == prev_i:
                            targetwise_pred_scores[-1].append(p)
                        else:
                            targetwise_pred_scores.append([p])
                        prev_i = i
                for targetwise_pred_score in targetwise_pred_scores:
                    anchorwise_preds.append(np.mean(targetwise_pred_score))
            preds.append(anchorwise_preds)
        return preds
    # postprocess
    import numpy as np
    import pandas as pd

    import re
    import nltk
    from nltk.stem import PorterStemmer
    from nltk.stem import WordNetLemmatizer

    class PostProcess:
        def __init__(self, df, options={}):
            self.options = {
                'fit_near_score_th': 0.005,
                'normalizer': ('lemmatizer') # 'stemmer'
            }
            self.options.update(options)
            self.df = df.copy()
            self.predictions = []

        def add_predection(self, pred):
            # shape => (36, 1)
            self.predictions.append(pred.reshape(-1, 1))

        def nanmean(self, preds):
            return np.nanmean(preds, axis=0).reshape(-1)

        def equal_score_1(self, preds):
            """
            単純に anchor == target は score 1.0 にする
            """
            df = self.df.copy()
            df['preds'] = preds
            df.loc[df.anchor == df.target, ['preds']] = 1.0
            return df['preds'].values

        def normalize_score_1(self, preds):
            """
            normalizeした結果 == なら score を 1.0にする
            """
            df = self.df.copy()
            normalizer_options = self.options['normalizer']
            lemmatizer = WordNetLemmatizer()
            ps = PorterStemmer()
            use_lemmatizer = 'lemmatizer' in normalizer_options
            use_stemmer = 'stemmer' in normalizer_options
            def normalize(word):
                w = word.lower()
                if use_lemmatizer:
                    w = lemmatizer.lemmatize(w)
                if use_stemmer:
                    w = ps.stem(w)
                return w
            def normalize_words(word):
                words = re.split(r'\s+', word)
                words = [normalize(t) for t in words]
                return ' '.join(words)
            an = df.anchor.map(normalize_words)
            tn = df.target.map(normalize_words)
            df['preds'] = preds
            df.loc[an == tn, ['preds']] = 1.0
            return df['preds'].values

        def fit_near_score(self, preds):
            fit_n = self.options['fit_near_score_th']
            def fit_score(pred):
                if pred <= fit_n:
                    return 0.0
                elif pred >= (0.25 - fit_n) and pred <= (0.25 + fit_n):
                    return 0.25
                elif pred >= (0.5 - fit_n) and pred <= (0.5 + fit_n):
                    return 0.5
                elif pred >= (0.75 - fit_n) and pred <= (0.75 + fit_n):
                    return 0.75
                elif pred >= (1 - fit_n):
                    return 1.0
                return pred
            vect = np.vectorize(fit_score)
            return vect(preds)

        def get_score(self, apply_funcs=['nanmean', 'equal_score_1', 'normalize_score_1', 'fit_near_score']):
            preds = self.predictions
            for funcname in apply_funcs:
                fn = getattr(self, funcname)
                preds = fn(preds)
            return preds.reshape(-1, 1) # 提出形式に合わせる


    post_process = PostProcess(df=exploded_test)
    # predictions
    from functools import reduce

    test_dataset = TestDataset(CFG, auged_test)
    test_loader = DataLoader(test_dataset,
                             batch_size=CFG.batch_size,
                             shuffle=False,
                             num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
    for fold in CFG.trn_fold:
        model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False, n_vocabs=len(CFG.tokenizer))
        state = torch.load(CFG.path+f"{CFG.ckpt_name}_fold{fold}_best.pth",
                           map_location=torch.device('cpu'))
        model.load_state_dict(state['model'])
        prediction = inference_fn(test_loader, model, device)
        prediction = np.array(reduce(lambda a,b: a+b, prediction))
        post_process.add_predection(prediction)
        del model, state, prediction; gc.collect()
        torch.cuda.empty_cache()
    predictions = post_process.get_score()
    # Submission
    exploded_test['score']=predictions
    exploded_test = exploded_test.groupby('id').mean().reset_index() # aggregate auged samples
    submission_099=exploded_test[['id','score']].sort_index()
    display(submission_099.head())
    submission_099[['id','score']].to_csv('submission.csv',index=False)
    return submission_099

In [ ]:
# 108
def exp108():
    # ====================================================
    # CFG
    # ====================================================
    class CFG:
        num_workers=0
        path="../input/pppm-exp108/"
        config_path=path+'config.pth'
        model="anferico/bert-for-patents"
        ckpt_name="bert_for_patents"
        batch_size=16
        target_size=1
        max_len=480
        seed=42
        n_fold=4
    #     trn_fold=[0]
        tar_token_id = 39859
        tar_token = '[TAR]'
        trn_fold=[0, 1, 2, 3]
        n_augs = 2

    # Library
    # ====================================================
    # Library
    # ====================================================
    import os
    import gc
    import re
    import ast
    import sys
    import copy
    import json
    import time
    import math
    import shutil
    import string
    import pickle
    import random
    import joblib
    import itertools
    from pathlib import Path
    import warnings
    warnings.filterwarnings("ignore")

    import scipy as sp
    import numpy as np
    import pandas as pd
    pd.set_option('display.max_rows', 500)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.width', 1000)
    from tqdm.auto import tqdm
    from sklearn.metrics import f1_score
    from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

    import torch
    print(f"torch.__version__: {torch.__version__}")
    import torch.nn as nn
    from torch.nn import Parameter
    import torch.nn.functional as F
    from torch.optim import Adam, SGD, AdamW
    from torch.utils.data import DataLoader, Dataset
    from torch import inference_mode

    # os.system('pip uninstall -y transformers')
    # os.system('pip uninstall -y tokenizers')
    # os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheel-20220531 transformers')
    # os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheel-20220531 tokenizers')

    import tokenizers
    import transformers
    print(f"tokenizers.__version__: {tokenizers.__version__}")
    print(f"transformers.__version__: {transformers.__version__}")
    from transformers import AutoTokenizer, AutoModel, AutoConfig
    from transformers import DataCollatorWithPadding
    from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
    %env TOKENIZERS_PARALLELISM=false #true

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Utils
    # ====================================================
    # Utils
    # ====================================================
    def get_score(y_true, y_pred):
        score = sp.stats.pearsonr(y_true, y_pred)[0]
        return score


    def get_logger(filename=OUTPUT_DIR+'train'):
        from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
        logger = getLogger(__name__)
        logger.setLevel(INFO)
        handler1 = StreamHandler()
        handler1.setFormatter(Formatter("%(message)s"))
        handler2 = FileHandler(filename=f"{filename}.log")
        handler2.setFormatter(Formatter("%(message)s"))
        logger.addHandler(handler1)
        logger.addHandler(handler2)
        return logger

    LOGGER = get_logger()

    def seed_everything(seed=42):
        random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True

    seed_everything(seed=42)
    # OOF
    oof_df = pd.read_pickle(CFG.path+'oof_df.pkl')
    labels = oof_df['score'].values
    preds = oof_df['pred'].values
    score = get_score(labels, preds)
    LOGGER.info(f'CV Score: {score:<.4f}')
    # Data Loading
    # ====================================================
    # Data Loading
    # ====================================================
    # test = pd.read_csv(INPUT_DIR+'train.csv').sample(n=100)
    test = pd.read_csv(INPUT_DIR+'test.csv')
    submission = pd.read_csv(INPUT_DIR+'sample_submission.csv')
    print(f"test.shape: {test.shape}")
    print(f"submission.shape: {submission.shape}")
    display(test.head())
    display(submission.head())
    # ====================================================
    # CPC Data
    # ====================================================
    cpc_texts = torch.load(CFG.path+"cpc_texts.pth")
    test['context_text'] = test['context'].map(cpc_texts)
    display(test.head())
    anchor_context_grouped_target = test.groupby(['anchor', 'context'])['target'].apply(list)
    anchor_context_grouped_id = test.groupby(['anchor', 'context'])['id'].apply(list)
    i = pd.DataFrame(anchor_context_grouped_id).reset_index()
    t = pd.DataFrame(anchor_context_grouped_target).reset_index()
    test = t.merge(i, on=['anchor', 'context'])
    test['context_text'] = test['context'].map(cpc_texts)
    test = test.rename(columns={'target': 'targets', 'id': 'ids'})
    test['n_ids'] = test['ids'].map(len)
    test.head()
    from functools import reduce

    def cat_ids(x) -> str:
        return reduce(lambda a,b: a+b, x)
    import random 

    cat_ids_sets = set(test['ids'].map(cat_ids))
    auged_test = test.copy()
    auged_rows = []
    for a in range(CFG.n_augs):
        for _,r in test.iterrows():
            if r.n_ids == 1:
                continue
            indices = list(range(r.n_ids))
            random.shuffle(indices)
            if cat_ids(list(np.array(r.ids)[indices])) in cat_ids_sets:
                continue
            auged_r = r.copy()
            auged_r.targets = list(np.array(auged_r.targets)[indices])
            auged_r.ids = list(np.array(auged_r.ids)[indices])
            cat_ids_sets.add(cat_ids(auged_r.ids))
            auged_rows.append(auged_r)
    auged_test = test.append(
        pd.DataFrame(auged_rows, columns=test.columns).reset_index(drop=True))
    exploded_test = auged_test.explode(['targets', 'ids']).rename(columns={
            'targets': 'target',
            'ids': 'id',
        }).reset_index(drop=True)
    # tokenizer
    # ====================================================
    # tokenizer
    # ====================================================
    CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')
    assert CFG.tokenizer.encode(CFG.tar_token, add_special_tokens=False)[0] == CFG.tar_token_id
    assert CFG.tokenizer.decode(CFG.tar_token_id) == CFG.tar_token

    setattr(CFG.tokenizer, 'tar_token', CFG.tar_token)
    setattr(CFG.tokenizer, 'tar_token_id', CFG.tar_token_id)
    # Dataset
    # ====================================================
    # Dataset
    # ====================================================

    class TestDataset(Dataset):
        def __init__(self, cfg, df):
            self.cfg = cfg
            self.anchors = df['anchor'].values
            self.target_lists = df['targets'].values
            self.id_lists = df['ids'].values
            self.context_texts = df['context_text'].values
            self.max_len = cfg.max_len
            self.tokenizer = cfg.tokenizer

        def __len__(self):
            return len(self.id_lists)

        def __getitem__(self, item):
            target_mask = np.zeros(self.max_len)
            targets = np.array(self.target_lists[item])

            text = ''
            text += self.tokenizer.cls_token
            text += self.anchors[item]
            text += self.tokenizer.sep_token
            for target in targets:
                text += target + self.tokenizer.tar_token
            text += self.context_texts[item] + self.tokenizer.sep_token

            encoded = self.tokenizer(
                text,
                max_length = self.max_len,
                padding='max_length',
                add_special_tokens=False,
                truncation=True
            )

            cnt_tar = 0
            cnt_sep = 0
            nth_target = -1
            prev_i = -1

            for i, input_id in enumerate(encoded['input_ids']):
                if input_id == self.tokenizer.tar_token_id:
                    cnt_tar += 1
                    if cnt_tar == len(targets):
                        break
                if input_id == self.tokenizer.sep_token_id:
                    cnt_sep += 1

                if cnt_sep == 1 and input_id not in [self.tokenizer.pad_token_id, self.tokenizer.sep_token_id, self.tokenizer.tar_token_id]:
                    if (i-prev_i) > 1:
                        nth_target += 1
                    target_mask[i] = 1
                    prev_i = i

            for k,v in encoded.items():
                encoded[k] = torch.tensor(v, dtype=torch.long)

            return encoded, target_mask
    # Model
    # ====================================================
    # Model
    # ====================================================
    class CustomModel(nn.Module):
        def __init__(self, cfg, config_path=None, pretrained=False, n_vocabs=0):
            super().__init__()
            self.cfg = cfg
            if config_path is None:
                self.config = AutoConfig.from_pretrained(
                    cfg.model, output_hidden_states=True)
            else:
                self.config = torch.load(config_path)
            if pretrained:
                self.model = AutoModel.from_pretrained(
                    cfg.model, config=self.config)
            else:
                self.model = AutoModel.from_config(self.config)
            self.model.resize_token_embeddings(n_vocabs)
            self.fc = nn.Linear(self.config.hidden_size, 1)

        def feature(self, inputs):
            outputs = self.model(**inputs)
            last_hidden_states = outputs[0]
            return last_hidden_states

        def forward(self, inputs):
            feature = self.feature(inputs)
            output = self.fc(feature).squeeze(-1)
            return output
    # inference
    # ====================================================
    # inference
    # ====================================================
    @inference_mode()
    def inference_fn(test_loader, model, device):
        preds = []
        model.eval()
        model.to(device)
        for inputs, target_masks in test_loader:
            for k, v in inputs.items():
                inputs[k] = v.to(device)
            y_preds = model(inputs)
            y_preds = y_preds.sigmoid().to('cpu').numpy()

            anchorwise_preds = []
            for pred, target_mask, in zip(y_preds, target_masks):
                prev_i = -1
                targetwise_pred_scores = []
                for i, (p, tm) in enumerate(zip(pred, target_mask)):
                    if tm != 0:
                        if i-1 == prev_i:
                            targetwise_pred_scores[-1].append(p)
                        else:
                            targetwise_pred_scores.append([p])
                        prev_i = i
                for targetwise_pred_score in targetwise_pred_scores:
                    anchorwise_preds.append(np.mean(targetwise_pred_score))
            preds.append(anchorwise_preds)
        return preds
    # postprocess
    import numpy as np
    import pandas as pd

    import re
    import nltk
    from nltk.stem import PorterStemmer
    from nltk.stem import WordNetLemmatizer

    class PostProcess:
        def __init__(self, df, options={}):
            self.options = {
                'fit_near_score_th': 0.005,
                'normalizer': ('lemmatizer') # 'stemmer'
            }
            self.options.update(options)
            self.df = df.copy()
            self.predictions = []

        def add_predection(self, pred):
            # shape => (36, 1)
            self.predictions.append(pred.reshape(-1, 1))

        def nanmean(self, preds):
            return np.nanmean(preds, axis=0).reshape(-1)

        def equal_score_1(self, preds):
            """
            単純に anchor == target は score 1.0 にする
            """
            df = self.df.copy()
            df['preds'] = preds
            df.loc[df.anchor == df.target, ['preds']] = 1.0
            return df['preds'].values

        def normalize_score_1(self, preds):
            """
            normalizeした結果 == なら score を 1.0にする
            """
            df = self.df.copy()
            normalizer_options = self.options['normalizer']
            lemmatizer = WordNetLemmatizer()
            ps = PorterStemmer()
            use_lemmatizer = 'lemmatizer' in normalizer_options
            use_stemmer = 'stemmer' in normalizer_options
            def normalize(word):
                w = word.lower()
                if use_lemmatizer:
                    w = lemmatizer.lemmatize(w)
                if use_stemmer:
                    w = ps.stem(w)
                return w
            def normalize_words(word):
                words = re.split(r'\s+', word)
                words = [normalize(t) for t in words]
                return ' '.join(words)
            an = df.anchor.map(normalize_words)
            tn = df.target.map(normalize_words)
            df['preds'] = preds
            df.loc[an == tn, ['preds']] = 1.0
            return df['preds'].values

        def fit_near_score(self, preds):
            fit_n = self.options['fit_near_score_th']
            def fit_score(pred):
                if pred <= fit_n:
                    return 0.0
                elif pred >= (0.25 - fit_n) and pred <= (0.25 + fit_n):
                    return 0.25
                elif pred >= (0.5 - fit_n) and pred <= (0.5 + fit_n):
                    return 0.5
                elif pred >= (0.75 - fit_n) and pred <= (0.75 + fit_n):
                    return 0.75
                elif pred >= (1 - fit_n):
                    return 1.0
                return pred
            vect = np.vectorize(fit_score)
            return vect(preds)

        def get_score(self, apply_funcs=['nanmean', 'equal_score_1', 'normalize_score_1', 'fit_near_score']):
            preds = self.predictions
            for funcname in apply_funcs:
                fn = getattr(self, funcname)
                preds = fn(preds)
            return preds.reshape(-1, 1) # 提出形式に合わせる


    post_process = PostProcess(df=exploded_test)
    # predictions
    from functools import reduce

    test_dataset = TestDataset(CFG, auged_test)
    test_loader = DataLoader(test_dataset,
                             batch_size=CFG.batch_size,
                             shuffle=False,
                             num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
    for fold in CFG.trn_fold:
        model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False, n_vocabs=len(CFG.tokenizer))
        state = torch.load(CFG.path+f"{CFG.ckpt_name}_fold{fold}_best.pth",
                           map_location=torch.device('cpu'))
        model.load_state_dict(state['model'])
        prediction = inference_fn(test_loader, model, device)
        prediction = np.array(reduce(lambda a,b: a+b, prediction))
        post_process.add_predection(prediction)
        del model, state, prediction; gc.collect()
        torch.cuda.empty_cache()
    predictions = post_process.get_score()
    # Submission
    exploded_test['score']=predictions
    exploded_test = exploded_test.groupby('id').mean().reset_index() # aggregate auged samples
    submission_108=exploded_test[['id','score']].sort_index()
    display(submission_108.head())
    submission_108[['id','score']].to_csv('submission.csv',index=False)
    return submission_108

In [ ]:
# 124
def exp124():
    # ====================================================
    # CFG
    # ====================================================
    class CFG:
        num_workers=0
        path="../input/pppm-exp124/"
        config_path=path+'config.pth'
        model="MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli"
        ckpt_name="DeBERTa-v3-large-mnli-fever-anli-ling-wanli"
        batch_size=16
        target_size=1
        max_len=512
        seed=42
        n_fold=4
        tar_token_id = 128001
        tar_token = '[TAR]'
        trn_fold=[0, 1, 2, 3]
        n_augs = 2

    # Library
    # ====================================================
    # Library
    # ====================================================
    import os
    import gc
    import re
    import ast
    import sys
    import copy
    import json
    import time
    import math
    import shutil
    import string
    import pickle
    import random
    import joblib
    import itertools
    from pathlib import Path
    import warnings
    warnings.filterwarnings("ignore")

    import scipy as sp
    import numpy as np
    import pandas as pd
    pd.set_option('display.max_rows', 500)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.width', 1000)
    from tqdm.auto import tqdm
    from sklearn.metrics import f1_score
    from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

    import torch
    print(f"torch.__version__: {torch.__version__}")
    import torch.nn as nn
    from torch.nn import Parameter
    import torch.nn.functional as F
    from torch.optim import Adam, SGD, AdamW
    from torch.utils.data import DataLoader, Dataset
    from torch import inference_mode

    # os.system('pip uninstall -y transformers')
    # os.system('pip uninstall -y tokenizers')
    # os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheel-20220531 transformers')
    # os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheel-20220531 tokenizers')

    import tokenizers
    import transformers
    print(f"tokenizers.__version__: {tokenizers.__version__}")
    print(f"transformers.__version__: {transformers.__version__}")
    from transformers import AutoTokenizer, AutoModel, AutoConfig
    from transformers import DataCollatorWithPadding
    from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
    %env TOKENIZERS_PARALLELISM=false #true

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Utils
    # ====================================================
    # Utils
    # ====================================================
    def get_score(y_true, y_pred):
        score = sp.stats.pearsonr(y_true, y_pred)[0]
        return score


    def get_logger(filename=OUTPUT_DIR+'train'):
        from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
        logger = getLogger(__name__)
        logger.setLevel(INFO)
        handler1 = StreamHandler()
        handler1.setFormatter(Formatter("%(message)s"))
        handler2 = FileHandler(filename=f"{filename}.log")
        handler2.setFormatter(Formatter("%(message)s"))
        logger.addHandler(handler1)
        logger.addHandler(handler2)
        return logger

    LOGGER = get_logger()

    def seed_everything(seed=42):
        random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True

    seed_everything(seed=42)
    # OOF
    oof_df = pd.read_pickle(CFG.path+'oof_df.pkl')
    labels = oof_df['score'].values
    preds = oof_df['pred'].values
    score = get_score(labels, preds)
    LOGGER.info(f'CV Score: {score:<.4f}')
    # Data Loading
    # ====================================================
    # Data Loading
    # ====================================================
    # test = pd.read_csv(INPUT_DIR+'train.csv').sample(n=100)
    test = pd.read_csv(INPUT_DIR+'test.csv')
    submission = pd.read_csv(INPUT_DIR+'sample_submission.csv')
    print(f"test.shape: {test.shape}")
    print(f"submission.shape: {submission.shape}")
    display(test.head())
    display(submission.head())
    # ====================================================
    # CPC Data
    # ====================================================
    cpc_texts = torch.load(CFG.path+"cpc_texts.pth")
    test['context_text'] = test['context'].map(cpc_texts)
    display(test.head())
    anchor_context_grouped_target = test.groupby(['anchor', 'context'])['target'].apply(list)
    anchor_context_grouped_id = test.groupby(['anchor', 'context'])['id'].apply(list)
    i = pd.DataFrame(anchor_context_grouped_id).reset_index()
    t = pd.DataFrame(anchor_context_grouped_target).reset_index()
    test = t.merge(i, on=['anchor', 'context'])
    test['context_text'] = test['context'].map(cpc_texts)
    test = test.rename(columns={'target': 'targets', 'id': 'ids'})
    test['n_ids'] = test['ids'].map(len)
    test.head()
    from functools import reduce

    def cat_ids(x) -> str:
        return reduce(lambda a,b: a+b, x)
    import random 

    cat_ids_sets = set(test['ids'].map(cat_ids))
    auged_test = test.copy()
    auged_rows = []
    for a in range(CFG.n_augs):
        for _,r in test.iterrows():
            if r.n_ids == 1:
                continue
            indices = list(range(r.n_ids))
            random.shuffle(indices)
            if cat_ids(list(np.array(r.ids)[indices])) in cat_ids_sets:
                continue
            auged_r = r.copy()
            auged_r.targets = list(np.array(auged_r.targets)[indices])
            auged_r.ids = list(np.array(auged_r.ids)[indices])
            cat_ids_sets.add(cat_ids(auged_r.ids))
            auged_rows.append(auged_r)
    auged_test = test.append(
        pd.DataFrame(auged_rows, columns=test.columns).reset_index(drop=True))
    exploded_test = auged_test.explode(['targets', 'ids']).rename(columns={
            'targets': 'target',
            'ids': 'id',
        }).reset_index(drop=True)
    # tokenizer
    # ====================================================
    # tokenizer
    # ====================================================
    CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')
    assert CFG.tokenizer.encode(CFG.tar_token, add_special_tokens=False)[0] == CFG.tar_token_id
    assert CFG.tokenizer.decode(CFG.tar_token_id) == CFG.tar_token

    setattr(CFG.tokenizer, 'tar_token', CFG.tar_token)
    setattr(CFG.tokenizer, 'tar_token_id', CFG.tar_token_id)
    # Dataset
    # ====================================================
    # Dataset
    # ====================================================

    class TestDataset(Dataset):
        def __init__(self, cfg, df):
            self.cfg = cfg
            self.anchors = df['anchor'].values
            self.target_lists = df['targets'].values
            self.id_lists = df['ids'].values
            self.context_texts = df['context_text'].values
            self.max_len = cfg.max_len
            self.tokenizer = cfg.tokenizer

        def __len__(self):
            return len(self.id_lists)

        def __getitem__(self, item):
            target_mask = np.zeros(self.max_len)
            targets = np.array(self.target_lists[item])

            text = ''
            text += self.tokenizer.cls_token
            text += self.anchors[item]
            text += self.tokenizer.sep_token
            for target in targets:
                text += target + self.tokenizer.tar_token
            text += self.context_texts[item] + self.tokenizer.sep_token

            encoded = self.tokenizer(
                text,
                max_length = self.max_len,
                padding='max_length',
                add_special_tokens=False,
                truncation=True
            )

            cnt_tar = 0
            cnt_sep = 0
            nth_target = -1
            prev_i = -1

            for i, input_id in enumerate(encoded['input_ids']):
                if input_id == self.tokenizer.tar_token_id:
                    cnt_tar += 1
                    if cnt_tar == len(targets):
                        break
                if input_id == self.tokenizer.sep_token_id:
                    cnt_sep += 1

                if cnt_sep == 1 and input_id not in [self.tokenizer.pad_token_id, self.tokenizer.sep_token_id, self.tokenizer.tar_token_id]:
                    if (i-prev_i) > 1:
                        nth_target += 1
                    target_mask[i] = 1
                    prev_i = i

            for k,v in encoded.items():
                encoded[k] = torch.tensor(v, dtype=torch.long)

            return encoded, target_mask
    # Model
    # ====================================================
    # Model
    # ====================================================
    class CustomModel(nn.Module):
        def __init__(self, cfg, config_path=None, pretrained=False, n_vocabs=0):
            super().__init__()
            self.cfg = cfg
            if config_path is None:
                self.config = AutoConfig.from_pretrained(
                    cfg.model, output_hidden_states=True)
            else:
                self.config = torch.load(config_path)
            if pretrained:
                self.model = AutoModel.from_pretrained(
                    cfg.model, config=self.config)
            else:
                self.model = AutoModel.from_config(self.config)
            self.model.resize_token_embeddings(n_vocabs)
            self.fc = nn.Linear(self.config.hidden_size, 1)

        def feature(self, inputs):
            outputs = self.model(**inputs)
            last_hidden_states = outputs[0]
            return last_hidden_states

        def forward(self, inputs):
            feature = self.feature(inputs)
            output = self.fc(feature).squeeze(-1)
            return output
    # inference
    # ====================================================
    # inference
    # ====================================================
    @inference_mode()
    def inference_fn(test_loader, model, device):
        preds = []
        model.eval()
        model.to(device)
        for inputs, target_masks in test_loader:
            for k, v in inputs.items():
                inputs[k] = v.to(device)
            y_preds = model(inputs)
            y_preds = y_preds.sigmoid().to('cpu').numpy()

            anchorwise_preds = []
            for pred, target_mask, in zip(y_preds, target_masks):
                prev_i = -1
                targetwise_pred_scores = []
                for i, (p, tm) in enumerate(zip(pred, target_mask)):
                    if tm != 0:
                        if i-1 == prev_i:
                            targetwise_pred_scores[-1].append(p)
                        else:
                            targetwise_pred_scores.append([p])
                        prev_i = i
                for targetwise_pred_score in targetwise_pred_scores:
                    anchorwise_preds.append(np.mean(targetwise_pred_score))
            preds.append(anchorwise_preds)
        return preds
    # postprocess
    import numpy as np
    import pandas as pd

    import re
    import nltk
    from nltk.stem import PorterStemmer
    from nltk.stem import WordNetLemmatizer

    class PostProcess:
        def __init__(self, df, options={}):
            self.options = {
                'fit_near_score_th': 0.005,
                'normalizer': ('lemmatizer') # 'stemmer'
            }
            self.options.update(options)
            self.df = df.copy()
            self.predictions = []

        def add_predection(self, pred):
            # shape => (36, 1)
            self.predictions.append(pred.reshape(-1, 1))

        def nanmean(self, preds):
            return np.nanmean(preds, axis=0).reshape(-1)

        def equal_score_1(self, preds):
            """
            単純に anchor == target は score 1.0 にする
            """
            df = self.df.copy()
            df['preds'] = preds
            df.loc[df.anchor == df.target, ['preds']] = 1.0
            return df['preds'].values

        def normalize_score_1(self, preds):
            """
            normalizeした結果 == なら score を 1.0にする
            """
            df = self.df.copy()
            normalizer_options = self.options['normalizer']
            lemmatizer = WordNetLemmatizer()
            ps = PorterStemmer()
            use_lemmatizer = 'lemmatizer' in normalizer_options
            use_stemmer = 'stemmer' in normalizer_options
            def normalize(word):
                w = word.lower()
                if use_lemmatizer:
                    w = lemmatizer.lemmatize(w)
                if use_stemmer:
                    w = ps.stem(w)
                return w
            def normalize_words(word):
                words = re.split(r'\s+', word)
                words = [normalize(t) for t in words]
                return ' '.join(words)
            an = df.anchor.map(normalize_words)
            tn = df.target.map(normalize_words)
            df['preds'] = preds
            df.loc[an == tn, ['preds']] = 1.0
            return df['preds'].values

        def fit_near_score(self, preds):
            fit_n = self.options['fit_near_score_th']
            def fit_score(pred):
                if pred <= fit_n:
                    return 0.0
                elif pred >= (0.25 - fit_n) and pred <= (0.25 + fit_n):
                    return 0.25
                elif pred >= (0.5 - fit_n) and pred <= (0.5 + fit_n):
                    return 0.5
                elif pred >= (0.75 - fit_n) and pred <= (0.75 + fit_n):
                    return 0.75
                elif pred >= (1 - fit_n):
                    return 1.0
                return pred
            vect = np.vectorize(fit_score)
            return vect(preds)

        def get_score(self, apply_funcs=['nanmean', 'equal_score_1', 'normalize_score_1', 'fit_near_score']):
            preds = self.predictions
            for funcname in apply_funcs:
                fn = getattr(self, funcname)
                preds = fn(preds)
            return preds.reshape(-1, 1) # 提出形式に合わせる


    post_process = PostProcess(df=exploded_test)
    # predictions
    from functools import reduce

    test_dataset = TestDataset(CFG, auged_test)
    test_loader = DataLoader(test_dataset,
                             batch_size=CFG.batch_size,
                             shuffle=False,
                             num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
    for fold in CFG.trn_fold:
        model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False, n_vocabs=len(CFG.tokenizer))
        state = torch.load(CFG.path+f"{CFG.ckpt_name}_fold{fold}_best.pth",
                           map_location=torch.device('cpu'))
        model.load_state_dict(state['model'])
        prediction = inference_fn(test_loader, model, device)
        prediction = np.array(reduce(lambda a,b: a+b, prediction))
        post_process.add_predection(prediction)
        del model, state, prediction; gc.collect()
        torch.cuda.empty_cache()
    predictions = post_process.get_score()
    # Submission
    exploded_test['score']=predictions
    exploded_test = exploded_test.groupby('id').mean().reset_index() # aggregate auged samples
    submission=exploded_test[['id','score']].sort_index()
    display(submission.head())
    submission[['id','score']].to_csv('submission.csv',index=False)
    return submission

In [ ]:
def exp115():
    class CFG:
        num_workers=0
        path="../input/pppm-exp115/"
        config_path=path+'config.pth'
        model="microsoft/deberta-v3-large"
        ckpt_name="deberta-v3-large"
        batch_size=32
        target_size=1
        max_len=160
        seed=42
        n_fold=4
        trn_fold=[0, 1, 2, 3]

        
    import os
    import gc
    import re
    import ast
    import sys
    import copy
    import json
    import time
    import math
    import shutil
    import string
    import pickle
    import random
    import joblib
    import itertools
    from pathlib import Path
    import warnings
    warnings.filterwarnings("ignore")

    import scipy as sp
    import numpy as np
    import pandas as pd
    pd.set_option('display.max_rows', 500)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.width', 1000)
    from tqdm.auto import tqdm
    from sklearn.metrics import f1_score
    from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

    import torch
    print(f"torch.__version__: {torch.__version__}")
    import torch.nn as nn
    from torch.nn import Parameter
    import torch.nn.functional as F
    from torch.optim import Adam, SGD, AdamW
    from torch.utils.data import DataLoader, Dataset
    from torch import inference_mode

    import tokenizers
    import transformers
    print(f"tokenizers.__version__: {tokenizers.__version__}")
    print(f"transformers.__version__: {transformers.__version__}")
    from transformers import AutoTokenizer, AutoModel, AutoConfig
    from transformers import DataCollatorWithPadding
    from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
    %env TOKENIZERS_PARALLELISM=false #true

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # ====================================================
    # Utils
    # ====================================================
    def get_score(y_true, y_pred):
        score = sp.stats.pearsonr(y_true, y_pred)[0]
        return score


    def get_logger(filename=OUTPUT_DIR+'train'):
        from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
        logger = getLogger(__name__)
        logger.setLevel(INFO)
        handler1 = StreamHandler()
        handler1.setFormatter(Formatter("%(message)s"))
        handler2 = FileHandler(filename=f"{filename}.log")
        handler2.setFormatter(Formatter("%(message)s"))
        logger.addHandler(handler1)
        logger.addHandler(handler2)
        return logger

    LOGGER = get_logger()

    def seed_everything(seed=42):
        random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True

    seed_everything(seed=42)

    oof_df = pd.read_pickle(CFG.path+'oof_df.pkl')
    labels = oof_df['score'].values
    preds = oof_df['pred'].values
    score = get_score(labels, preds)
    LOGGER.info(f'CV Score: {score:<.4f}')

    # ====================================================
    # Data Loading
    # ====================================================
    test = pd.read_csv(INPUT_DIR+'test.csv')
    submission = pd.read_csv(INPUT_DIR+'sample_submission.csv')
    print(f"test.shape: {test.shape}")
    print(f"submission.shape: {submission.shape}")
    display(test.head())
    display(submission.head())

    # ====================================================
    # CPC Data
    # ====================================================
    cpc_texts = torch.load(CFG.path+"cpc_texts.pth")
    test['context_text'] = test['context'].map(cpc_texts)
    display(test.head())

    test['text'] = test['anchor'] + '[SEP]' + test['target'] + '[SEP]'  + test['context_text']
    display(test.head())

    # ====================================================
    # tokenizer
    # ====================================================
    CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')

    # token_length付与
    test['token_length']=CFG.tokenizer(test['text'].to_list(), max_length=CFG.max_len, truncation=True, return_length=True)['length']
    test=test.sort_values('token_length')
    test.head()

    # ====================================================
    # Dataset
    # ====================================================

    class TestDataset(Dataset):
        def __init__(self, cfg, df):
            self.cfg = cfg
            self.texts = df['text'].values

        def __len__(self):
            return len(self.texts)

        def __getitem__(self, item):
            inputs = self.texts[item]
            return inputs

    def collate_fn(data: "list[str]"
                ) -> "dict[str,torch.Tensor]":
        '''DataLoaderで使用するcollate_fn. Datasetの出力を受け取り、tensorの形に加工する関数。
        ここでは、動的にpaddingを行うために使用している'''
        texts = data
        inputs = CFG.tokenizer(texts,
                            add_special_tokens=True,
                            max_length=CFG.max_len,
                            truncation=True,
                            return_offsets_mapping=False)
        data_collator = DataCollatorWithPadding(tokenizer=CFG.tokenizer)
        inputs = data_collator(inputs)
        return inputs

    # ====================================================
    # Model
    # ====================================================
    class CustomModel(nn.Module):
        def __init__(self, cfg, config_path=None, pretrained=False):
            super().__init__()
            self.cfg = cfg
            if config_path is None:
                self.config = AutoConfig.from_pretrained(
                    cfg.model, output_hidden_states=True)
            else:
                self.config = torch.load(config_path)
            if pretrained:
                self.model = AutoModel.from_pretrained(
                    cfg.model, config=self.config)
            else:
                self.model = AutoModel.from_config(self.config)
            self.fc = nn.Linear(self.config.hidden_size, self.cfg.target_size)
            self._init_weights(self.fc)

        def _init_weights(self, module):
            if isinstance(module, nn.Linear):
                module.weight.data.normal_(
                    mean=0.0, std=self.config.initializer_range)
                if module.bias is not None:
                    module.bias.data.zero_()
            elif isinstance(module, nn.Embedding):
                module.weight.data.normal_(
                    mean=0.0, std=self.config.initializer_range)
                if module.padding_idx is not None:
                    module.weight.data[module.padding_idx].zero_()
            elif isinstance(module, nn.LayerNorm):
                module.bias.data.zero_()
                module.weight.data.fill_(1.0)

        def feature(self, inputs):
            outputs = self.model(**inputs)
            # 最終層の出力 #(, max_len, hidden_size)
            last_hidden_states = outputs['last_hidden_state']
            # feature = torch.mean(last_hidden_states, 1)
            # ただ単に平均するんじゃなくて、どのhidden_sizeがどのくらい重要なのかで合計する
            feature = last_hidden_states[:, 0, :]  # (, hidden_size)
            return feature

        def forward(self, inputs):
            feature = self.feature(inputs)  # (, hidden_size)
            output = self.fc(feature)  # (,target_size)
            return output

    # ====================================================
    # inference
    # ====================================================
    @inference_mode()
    def inference_fn(test_loader, model, device):
        preds = []
        model.eval()
        model.to(device)
        tk0 = tqdm(test_loader, total=len(test_loader))
        for inputs in tk0:
            for k, v in inputs.items():
                inputs[k] = v.to(device)
            y_preds = model(inputs)
            preds.append(y_preds.sigmoid().to('cpu').numpy())
        predictions = np.concatenate(preds)
        return predictions

    # postprocess

    import numpy as np
    import pandas as pd

    import re
    import nltk
    # nltk.download('stopwords')
    # nltk.download('wordnet')
    # nltk.download('omw-1.4')
    from nltk.stem import PorterStemmer
    from nltk.stem import WordNetLemmatizer

    class PostProcess:
        def __init__(self, df, options={}):
            self.options = {
                'fit_near_score_th': 0.005,
                'normalizer': ('lemmatizer') # 'stemmer'
            }
            self.options.update(options)
            self.df = df.copy()
            self.predictions = []
        
        def add_predection(self, pred):
            # shape => (36, 1)
            self.predictions.append(pred.reshape(-1, 1))

        def nanmean(self, preds):
            return np.nanmean(preds, axis=0).reshape(-1)
        
        def equal_score_1(self, preds):
            """
            単純に anchor == target は score 1.0 にする
            """
            df = self.df.copy()
            df['preds'] = preds
            df.loc[df.anchor == df.target, ['preds']] = 1.0
            return df['preds'].values
        
        def normalize_score_1(self, preds):
            """
            normalizeした結果 == なら score を 1.0にする
            """
            df = self.df.copy()
            normalizer_options = self.options['normalizer']
            lemmatizer = WordNetLemmatizer()
            ps = PorterStemmer()
            use_lemmatizer = 'lemmatizer' in normalizer_options
            use_stemmer = 'stemmer' in normalizer_options
            def normalize(word):
                w = word.lower()
                if use_lemmatizer:
                    w = lemmatizer.lemmatize(w)
                if use_stemmer:
                    w = ps.stem(w)
                return w
            def normalize_words(word):
                words = re.split(r'\s+', word)
                words = [normalize(t) for t in words]
                return ' '.join(words)
            an = df.anchor.map(normalize_words)
            tn = df.target.map(normalize_words)
            df['preds'] = preds
            df.loc[an == tn, ['preds']] = 1.0
            return df['preds'].values
        
        def fit_near_score(self, preds):
            fit_n = self.options['fit_near_score_th']
            def fit_score(pred):
                if pred <= fit_n:
                    return 0.0
                elif pred >= (0.25 - fit_n) and pred <= (0.25 + fit_n):
                    return 0.25
                elif pred >= (0.5 - fit_n) and pred <= (0.5 + fit_n):
                    return 0.5
                elif pred >= (0.75 - fit_n) and pred <= (0.75 + fit_n):
                    return 0.75
                elif pred >= (1 - fit_n):
                    return 1.0
                return pred
            vect = np.vectorize(fit_score)
            return vect(preds)

        def get_score(self, apply_funcs=['nanmean', 'equal_score_1', 'normalize_score_1', 'fit_near_score']):
            preds = self.predictions
            for funcname in apply_funcs:
                fn = getattr(self, funcname)
                preds = fn(preds)
            return preds.reshape(-1, 1) # 提出形式に合わせる
    post_process = PostProcess(df=test)

    # exp-070 prediction
    test_dataset = TestDataset(CFG, test)
    test_loader = DataLoader(test_dataset,
                            batch_size=CFG.batch_size,
                            shuffle=False,
                            num_workers=CFG.num_workers, pin_memory=True, drop_last=False,
                            collate_fn=collate_fn)
    # predictions = []
    for fold in CFG.trn_fold:
        model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
        state = torch.load(CFG.path+f"{CFG.ckpt_name}_fold{fold}_best.pth",
                        map_location=torch.device('cpu'))
        model.load_state_dict(state['model'])
        prediction = inference_fn(test_loader, model, device)
        post_process.add_predection(prediction)
        # predictions.append(prediction)
        del model, state, prediction; gc.collect()
        torch.cuda.empty_cache()
    # predictions = np.nanmean(predictions, axis=0)
    preds = post_process.get_score()
    test['score'] = preds
    return test[['id','score']]


In [ ]:
def exp114():
    # ====================================================
    # CFG
    # ====================================================
    class CFG:
        num_workers=2
        path="../input/pppm-exp114/"
        config_path=path+'config.pth'
        model="microsoft/deberta-v3-large"
        ckpt_name="deberta-v3-large"
        batch_size=32
        fc_dropout=0.2
        target_size=1
        max_len=133
        seed=42
        n_fold=4
        trn_fold=[0, 1, 2, 3]
    # Utils
    import os
    import gc
    import re
    import ast
    import sys
    import copy
    import json
    import time
    import math
    import shutil
    import string
    import pickle
    import random
    import joblib
    import itertools
    from pathlib import Path
    import warnings
    warnings.filterwarnings("ignore")

    import scipy as sp
    import numpy as np
    import pandas as pd
    pd.set_option('display.max_rows', 500)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.width', 1000)
    from tqdm.auto import tqdm
    from sklearn.metrics import f1_score
    from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

    import torch
    print(f"torch.__version__: {torch.__version__}")
    import torch.nn as nn
    from torch.nn import Parameter
    import torch.nn.functional as F
    from torch.optim import Adam, SGD, AdamW
    from torch.utils.data import DataLoader, Dataset
    from torch import inference_mode

    import tokenizers
    import transformers
    print(f"tokenizers.__version__: {tokenizers.__version__}")
    print(f"transformers.__version__: {transformers.__version__}")
    from transformers import AutoTokenizer, AutoModel, AutoConfig
    from transformers import DataCollatorWithPadding
    from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
    %env TOKENIZERS_PARALLELISM=false #true

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    
    # ====================================================
    # Utils
    # ====================================================
    def get_score(y_true, y_pred):
        score = sp.stats.pearsonr(y_true, y_pred)[0]
        return score


    def get_logger(filename=OUTPUT_DIR+'train'):
        from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
        logger = getLogger(__name__)
        logger.setLevel(INFO)
        handler1 = StreamHandler()
        handler1.setFormatter(Formatter("%(message)s"))
        handler2 = FileHandler(filename=f"{filename}.log")
        handler2.setFormatter(Formatter("%(message)s"))
        logger.addHandler(handler1)
        logger.addHandler(handler2)
        return logger

    LOGGER = get_logger()

    def seed_everything(seed=42):
        random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        
    seed_everything(seed=42)
    # OOF
    oof_df = pd.read_pickle(CFG.path+'oof_df.pkl')
    labels = oof_df['score'].values
    preds = oof_df['pred'].values
    score = get_score(labels, preds)
    LOGGER.info(f'CV Score: {score:<.4f}')
    # Data Loading
    # ====================================================
    # Data Loading
    # ====================================================
    test = pd.read_csv(INPUT_DIR+'test.csv')
    submission = pd.read_csv(INPUT_DIR+'sample_submission.csv')
    print(f"test.shape: {test.shape}")
    print(f"submission.shape: {submission.shape}")
    display(test.head())
    display(submission.head())
    # ====================================================
    # CPC Data
    # ====================================================
    cpc_texts = torch.load(CFG.path+"cpc_texts.pth")
    test['context_text'] = test['context'].map(cpc_texts)
    display(test.head())
    test['text'] = test['anchor'] + '[SEP]' + test['target'] + '[SEP]'  + test['context_text']
    display(test.head())
    # tokenizer
    # ====================================================
    # tokenizer
    # ====================================================
    CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')

    # token_length付与
    test['token_length']=CFG.tokenizer(test['text'].to_list(), max_length=CFG.max_len, truncation=True, return_length=True)['length']
    test=test.sort_values('token_length')
    test.head()
    # Dataset
    # ====================================================
    # Dataset
    # ====================================================
    # def prepare_input(cfg, text):
    #     inputs = cfg.tokenizer(text,
    #                            add_special_tokens=True,
    #                            max_length=cfg.max_len,
    #                            padding="max_length",
    #                            return_offsets_mapping=False)
    #     for k, v in inputs.items():
    #         inputs[k] = torch.tensor(v, dtype=torch.long)
    #     return inputs


    class TestDataset(Dataset):
        def __init__(self, cfg, df):
            self.cfg = cfg
            self.texts = df['text'].values

        def __len__(self):
            return len(self.texts)

        def __getitem__(self, item):
    #         inputs = prepare_input(self.cfg, self.texts[item])
            inputs = self.texts[item]
            return inputs

    def collate_fn(data: "list[str]"
                ) -> "dict[str,torch.Tensor]":
        '''DataLoaderで使用するcollate_fn. Datasetの出力を受け取り、tensorの形に加工する関数。
        ここでは、動的にpaddingを行うために使用している'''
        texts = data
        inputs = CFG.tokenizer(texts,
                            add_special_tokens=True,
                            max_length=CFG.max_len,
                            truncation=True,
                            return_offsets_mapping=False)
        data_collator = DataCollatorWithPadding(tokenizer=CFG.tokenizer)
        inputs = data_collator(inputs)
        return inputs
    # Model
    # ====================================================
    # Model
    # ====================================================
    class CustomModel(nn.Module):
        def __init__(self, cfg, config_path=None, pretrained=False):
            super().__init__()
            self.cfg = cfg
            if config_path is None:
                self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
            else:
                self.config = torch.load(config_path)
            if pretrained:
                self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
            else:
                self.model = AutoModel.from_config(self.config)
            self.fc_dropout = nn.Dropout(cfg.fc_dropout)
            self.fc = nn.Linear(self.config.hidden_size, self.cfg.target_size)
            self._init_weights(self.fc)
            self.attention = nn.Sequential(
                nn.Linear(self.config.hidden_size, 512),
                nn.Tanh(),
                nn.Linear(512, 1),
                nn.Softmax(dim=1)
            )
            self._init_weights(self.attention)
            
        def _init_weights(self, module):
            if isinstance(module, nn.Linear):
                module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
                if module.bias is not None:
                    module.bias.data.zero_()
            elif isinstance(module, nn.Embedding):
                module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
                if module.padding_idx is not None:
                    module.weight.data[module.padding_idx].zero_()
            elif isinstance(module, nn.LayerNorm):
                module.bias.data.zero_()
                module.weight.data.fill_(1.0)
            
        def feature(self, inputs):
            outputs = self.model(**inputs)
            last_hidden_states = outputs[0]
            # feature = torch.mean(last_hidden_states, 1)
            weights = self.attention(last_hidden_states)
            feature = torch.sum(weights * last_hidden_states, dim=1)
            return feature

        def forward(self, inputs):
            feature = self.feature(inputs)
            output = self.fc(self.fc_dropout(feature))
            return output
    # inference
    # ====================================================
    # inference
    # ====================================================
    @inference_mode()
    def inference_fn(test_loader, model, device):
        preds = []
        model.eval()
        model.to(device)
        tk0 = tqdm(test_loader, total=len(test_loader))
        for inputs in tk0:
            for k, v in inputs.items():
                inputs[k] = v.to(device)
            y_preds = model(inputs)
            preds.append(y_preds.sigmoid().to('cpu').numpy())
        predictions = np.concatenate(preds)
        return predictions
    test_dataset = TestDataset(CFG, test)
    test_loader = DataLoader(test_dataset,
                            batch_size=CFG.batch_size,
                            shuffle=False,
                            num_workers=CFG.num_workers, pin_memory=True, drop_last=False,
                            collate_fn=collate_fn)

    import numpy as np
    import pandas as pd

    import re
    import nltk
    # nltk.download('stopwords')
    # nltk.download('wordnet')
    # nltk.download('omw-1.4')
    from nltk.stem import PorterStemmer
    from nltk.stem import WordNetLemmatizer

    class PostProcess:
        def __init__(self, df, options={}):
            self.options = {
                'fit_near_score_th': 0.005,
                'normalizer': ('lemmatizer') # 'stemmer'
            }
            self.options.update(options)
            self.df = df.copy()
            self.predictions = []
        
        def add_predection(self, pred):
            # shape => (36, 1)
            self.predictions.append(pred.reshape(-1, 1))

        def nanmean(self, preds):
            return np.nanmean(preds, axis=0).reshape(-1)
        
        def equal_score_1(self, preds):
            """
            単純に anchor == target は score 1.0 にする
            """
            df = self.df.copy()
            df['preds'] = preds
            df.loc[df.anchor == df.target, ['preds']] = 1.0
            return df['preds'].values
        
        def normalize_score_1(self, preds):
            """
            normalizeした結果 == なら score を 1.0にする
            """
            df = self.df.copy()
            normalizer_options = self.options['normalizer']
            lemmatizer = WordNetLemmatizer()
            ps = PorterStemmer()
            use_lemmatizer = 'lemmatizer' in normalizer_options
            use_stemmer = 'stemmer' in normalizer_options
            def normalize(word):
                w = word.lower()
                if use_lemmatizer:
                    w = lemmatizer.lemmatize(w)
                if use_stemmer:
                    w = ps.stem(w)
                return w
            def normalize_words(word):
                words = re.split(r'\s+', word)
                words = [normalize(t) for t in words]
                return ' '.join(words)
            an = df.anchor.map(normalize_words)
            tn = df.target.map(normalize_words)
            df['preds'] = preds
            df.loc[an == tn, ['preds']] = 1.0
            return df['preds'].values
        
        def fit_near_score(self, preds):
            fit_n = self.options['fit_near_score_th']
            def fit_score(pred):
                if pred <= fit_n:
                    return 0.0
                elif pred >= (0.25 - fit_n) and pred <= (0.25 + fit_n):
                    return 0.25
                elif pred >= (0.5 - fit_n) and pred <= (0.5 + fit_n):
                    return 0.5
                elif pred >= (0.75 - fit_n) and pred <= (0.75 + fit_n):
                    return 0.75
                elif pred >= (1 - fit_n):
                    return 1.0
                return pred
            vect = np.vectorize(fit_score)
            return vect(preds)

        def get_score(self, apply_funcs=['nanmean', 'equal_score_1', 'normalize_score_1', 'fit_near_score']):
            preds = self.predictions
            for funcname in apply_funcs:
                fn = getattr(self, funcname)
                preds = fn(preds)
            return preds.reshape(-1, 1) # 提出形式に合わせる

    post_process = PostProcess(df=test)


    # predictions = []
    for fold in CFG.trn_fold:
        model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
        state = torch.load(CFG.path+f"{CFG.ckpt_name}_fold{fold}_best.pth",
                        map_location=torch.device('cpu'))
        model.load_state_dict(state['model'])
        prediction = inference_fn(test_loader, model, device)
        post_process.add_predection(prediction)
        # predictions.append(prediction)
        del model, state, prediction; gc.collect()
        torch.cuda.empty_cache()
    preds = post_process.get_score()
    test['score'] = preds
    return test[['id','score']]


In [ ]:
import pandas as pd

# submissions を結合する
submission_114 = exp114().sort_values('id').reset_index(drop=True)
submission_115 = exp115().sort_values('id').reset_index(drop=True)

submission_124 = exp124().sort_values('id').reset_index(drop=True)
submission_100 = exp100().sort_values('id').reset_index(drop=True)
submission_099 = exp099().sort_values('id').reset_index(drop=True)
submission_108 = exp108().sort_values('id').reset_index(drop=True)



In [ ]:
# 確認用
print('exp114')
display(submission_114[['id', 'score']].head())
print('exp115')
display(submission_115[['id', 'score']].head())

print('exp124')
display(submission_124[['id', 'score']].head())
print('exp100')
display(submission_100[['id', 'score']].head())
print('exp099')
display(submission_099[['id', 'score']].head())
print('exp108')
display(submission_108[['id', 'score']].head())




In [ ]:
import pandas as pd

# sunのid,scoreを結合前と比較すること
submission = pd.read_csv(INPUT_DIR+'sample_submission.csv').sort_values('id').reset_index(drop=True)
submission['score'] = submission_099.score * 0.1147 \
                    + submission_100.score * 0.1383 \
                    + submission_108.score * 0.2260 \
                    + submission_124.score * 0.2202 \
                    + submission_115.score * 0.1738 \
                    + submission_114.score * 0.1270
display(submission.head())
submission[['id','score']].to_csv('submission.csv',index=False)